Analysis for [insert your simulation]

[these are the residues I picked out for the 1hlr receptor, but feel free to change this according to the receptor you have]

[you will also need to make any changes to these residue numbers all throughout the file]

alpha C loop: 192-199

beta C loop: 192-200

alpha Cys loop: 131-145

beta Cys loop: 133-147

tip of alpha C loop (vicinal cysteines): 195-196

tip of beta C loop: 195-197

alpha backside tryptophan: 152

beta backside tryptophan: 154

SETUP ------------------------------------------------------------

In [1]:
# import necessary librarys so we have the functions that we need for analysis
# we assign them useful names for easy access
# import [library] as [name] #
import MDAnalysis as mda
from MDAnalysis.analysis import align, rms
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
# load in and create variables for .pdb and .xtc files
PDB = "[your pdb file; step3_input].pdb" # variable 'PDB' is the loaded PDB file
XTC = "[your trajectory file; trajout].xtc" # variable 'XTC' is the loaded trajectory file

In [ ]:
u = mda.Universe(PDB, XTC) # variable 'u' is the compiled structure and trajectory

print(u) # prints the universe
print(len(u.trajectory)) # shows the size of trajectory (in frames)

In [ ]:
# make more variables like 'u' so that we can manipulate them
mobile = mda.Universe(PDB, XTC) # variable 'mobile' compiles structure and trajectory
ref = mda.Universe(PDB, XTC) # variable 'ref' compiles structure and trajectory

RMSD ------------------------------------------------------------

In [ ]:
mobile.trajectory[-1] # set mobile trajectory to last frame
ref.trajectory[0] # set reference trajectory to first frame

mobile_ca = mobile.select_atoms('name CA') # select alpha carbons
ref_ca = ref.select_atoms('name CA') # select alpha carbons

# run rmsd with untreated (not aligned) frames for alpha carbons
rms.rmsd(mobile_ca.positions, ref_ca.positions, superposition = False)

output: RMSD of alpha carbons (untreated) = 

In [ ]:
# fit the whole trajectory to a reference structure
# so we are aligning the mobile trajectory to the reference trajectory
aligner = align.AlignTraj(mobile, ref, select='protein and name CA', in_memory = True)
aligner.run()

from here on out, the frames will be "treated". that is, the mobile trajectory is aligned with the reference trajectory. 

In [ ]:
mobile.trajectory[-1] # set mobile trajectory to last frame
ref.trajectory[0] # set reference trajectory to first frame

# run rmsd with treated frames for alpha carbons (should be a smaller value than before)
rms.rmsd(mobile_ca.positions, ref_ca.positions, superposition = False)

output: RMSD of alpha carbons (treated) = 

In [ ]:
mobile.trajectory[-1] # set mobile trajectory to last frame
output: RMSD of backbone = 2.63ref.trajectory[0] # set reference trajectory to first frame

# we can look at the relative movement of the backbone of the protein
# which, like the alpha carbons, allows us to look at the overall movement 
# of the protein
mobile_back = mobile.select_atoms('backbone') # variable where we select backbone
ref_back = ref.select_atoms('backbone') # variable where we select backbone

# run rmsd for backbone
rms.rmsd(mobile_back.positions, ref_back.positions, superposition = False)

output: RMSD of backbone = 

*when choosing residue numbers (resid) for the c loops, i looked at the specific numbers in Chimera using the Sequence Viewer. for segid, i have not been able to find a way to determine which subunit corresponds to which label in Chimera, so i used VMD to determine which segid each subunit is*

In [ ]:
mobile.trajectory[-1] # set mobile trajectory to last frame
ref.trajectory[0] # set reference trajectory to first frame

mobile_proa_c_loop = mobile.select_atoms('backbone and segid PROA and resid 192-199')
ref_proa_c_loop = ref.select_atoms('backbone and segid PROA and resid 192-199')

rms.rmsd(mobile_proa_c_loop.positions, ref_proa_c_loop.positions, superposition = False)

output: RMSD of C Loop on A subunit (a/b interface) = 

In [ ]:
mobile.trajectory[-1] # set mobile trajectory to last frame
ref.trajectory[0] # set reference trajectory to first frame

mobile_prod_c_loop = mobile.select_atoms('backbone and segid PROD and resid 192-199')
ref_prod_c_loop = ref.select_atoms('backbone and segid PROD and resid 192-199')

rms.rmsd(mobile_prod_c_loop.positions, ref_prod_c_loop.positions, superposition = False)

output: RMSD of C Loop on D subunit (a/b interface) = 

In [ ]:
mobile.trajectory[-1] # set mobile trajectory to last frame
ref.trajectory[0] # set reference trajectory to first frame

mobile_proe_c_loop = mobile.select_atoms('backbone and segid PROE and resid 192-200')
ref_proe_c_loop = ref.select_atoms('backbone and segid PROE and resid 192-200')

rms.rmsd(mobile_proe_c_loop.positions, ref_proe_c_loop.positions, superposition = False)

output: RMSD of C Loop on E subunit (b/a interface) = 

In [ ]:
mobile.trajectory[-1] # set mobile trajectory to last frame
ref.trajectory[0] # set reference trajectory to first frame

mobile_prob_c_loop = mobile.select_atoms('backbone and segid PROB and resid 192-200')
ref_prob_c_loop = ref.select_atoms('backbone and segid PROB and resid 192-200')

rms.rmsd(mobile_prob_c_loop.positions, ref_prob_c_loop.positions, superposition = False)

output: RMSD of C Loop of B subunit (b/b interface) = 

In [ ]:
mobile.trajectory[-1] # set mobile trajectory to last frame
ref.trajectory[0] # set reference trajectory to first frame

mobile_proc_c_loop = mobile.select_atoms('backbone and segid PROC and resid 192-200')
ref_proc_c_loop = ref.select_atoms('backbone and segid PROC and resid 192-200')

rms.rmsd(mobile_proc_c_loop.positions, ref_proc_c_loop.positions, superposition = False)

output: RMSD of C Loop of C subunit (b/a interface) = 

*rms.rmsd is a function that returns the RMSD between two coordinate sets, which in our case is the mobile coordinates and the reference coordinates. 
rms.RMSD (yes, capitalization matters here) performs RMSD analysis on a whole trajectory, which is what we want*

In [ ]:
# select the c loops of binding site subunits for RMSD analysis
C_loop_A = "backbone and segid PROA and resid 192-199"
C_loop_D = "backbone and segid PROD and resid 192-199"
C_loop_E = "backbone and segid PROE and resid 192-200"

R = rms.RMSD(mobile, ref, select='backbone', groupselections=[C_loop_A, C_loop_D, C_loop_E],ref_frame = 0)
R.run()

In [ ]:
df = pd.DataFrame(R.results.rmsd, columns=['Frame', 'Time (ps)', 'Backbone', 'C_loop_A', 'C_loop_D','C_loop_E']) # create table
df['Time (ns)']=(df['Time (ps)']/1000) # add a ns column
df # show table

In [ ]:
# create a plot of the data in the table above
ax = df.plot(x='Time (ns)',
             y=['C_loop_A', 'C_loop_D', 'C_loop_E', "Backbone"],
             label=['C loop ' r'$\alpha$/$\beta$ A', 'C loop ' r'$\alpha$/$\beta$ D', 'C loop ' r'$\beta$/$\alpha$ E', "Backbone"],
             color=['red', 'green', 'blue', 'yellow'])
ax.set_ylabel(r'RMSD ($\AA$)')

# save the plot to your computer
plt.savefig('RMSD_binding_sites.png')

In [ ]:
# select the c loops of all subunits for RMSD analysis
C_loop_A = "backbone and segid PROA and resid 192-199"
C_loop_D = "backbone and segid PROD and resid 192-199"
C_loop_E = "backbone and segid PROE and resid 192-200"
C_loop_B = "backbone and segid PROB and resid 192-200"
C_loop_C = "backbone and segid PROC and resid 192-200"

R = rms.RMSD(mobile, ref, select='backbone', groupselections=[C_loop_A, C_loop_D, C_loop_E, C_loop_B, C_loop_C],ref_frame = 0)
R.run()

In [ ]:
df = pd.DataFrame(R.results.rmsd, columns=['Frame', 'Time (ps)', 'Backbone', 'C_loop_A', 'C_loop_D','C_loop_E', 'C_loop_B', 'C_loop_C']) # create data table
df['Time (ns)']=(df['Time (ps)']/1000) # add time in ns column
df # show data table

In [ ]:
# create plot
ax = df.plot(x='Time (ns)',
             y=['C_loop_A', 'C_loop_D', 'C_loop_E', 'C_loop_B', 'C_loop_C', "Backbone"],
             label =['C loop ' r'$\alpha$/$\beta$ A', 'C loop ' r'$\alpha$/$\beta$ D', 'C loop ' r'$\beta$/$\alpha$ E', 'C loop ' r'$\beta$/$\beta$ B', 'C loop ' r'$\beta$/$\alpha$ C', "Backbone"],
             color=['red', 'green', 'blue', 'orange', 'purple', 'yellow'])
ax.set_ylabel(r'RMSD ($\AA$)')

# save plot to computer
plt.savefig('RMSD_all_sites.png')

In [ ]:
# select cys loop of all subunits rmsd analysis
Cys_loop_A = "backbone and segid PROA and resid 131-145"
Cys_loop_D = "backbone and segid PROD and resid 131-145"
Cys_loop_E = "backbone and segid PROE and resid 133-147"
Cys_loop_B = "backbone and segid PROB and resid 133-147"
Cys_loop_C = "backbone and segid PROC and resid 133-147"

R = rms.RMSD(mobile, ref, select='backbone', groupselections=[Cys_loop_A, Cys_loop_D, Cys_loop_E, Cys_loop_B, Cys_loop_C],ref_frame = 0)
R.run()

In [ ]:
# make data table
df = pd.DataFrame(R.results.rmsd, columns=['Frame', 'Time (ps)', 'Backbone', 'Cys_loop_A', 'Cys_loop_D','Cys_loop_E', 'Cys_loop_B', 'Cys_loop_C'])
df['Time (ns)']=(df['Time (ps)']/1000)
df

In [ ]:
# create plot
ax = df.plot(x='Time (ns)',
             y=['Cys_loop_A', 'Cys_loop_D', 'Cys_loop_E', 'Cys_loop_B', 'Cys_loop_C', "Backbone"],
             label =['Cys loop ' r'$\alpha$/$\beta$ A', 'Cys loop ' r'$\alpha$/$\beta$ D', 'Cys loop ' r'$\beta$/$\alpha$ E', 'Cys loop ' r'$\beta$/$\beta$ B', 'Cys loop ' r'$\beta$/$\alpha$ C', "Backbone"],
             color=['#F4A7A7', '#B0F4A7', '#62DDE9', '#F2C47B', '#D87BF2', 'yellow'])
ax.set_ylabel(r'RMSD ($\AA$)')
plt.savefig('RMSD_Cys_loop.png')

In [ ]:
# now we do rmsd for the cys and c loops but on the same subunit
# so this is rmsd of the cys and c loops of the proa subunit
Cys_loop_A = "backbone and segid PROA and resid 131-145"
C_loop_A = "backbone and segid PROA and resid 192-199"

R = rms.RMSD(mobile, ref, select='backbone', groupselections=[Cys_loop_A, C_loop_A],ref_frame = 0)
R.run()

In [ ]:
# make table
df = pd.DataFrame(R.results.rmsd, columns=['Frame', 'Time (ps)', 'Backbone', 'Cys_loop_A', 'C_loop_A'])
df['Time (ns)']=(df['Time (ps)']/1000)
df

In [ ]:
# make plot
ax = df.plot(x='Time (ns)',
             y=['Cys_loop_A', 'C_loop_A', "Backbone"],
             label =['Cys loop ' r'$\alpha$/$\beta$ A',  'C loop ' r'$\alpha$/$\beta$ A', "Backbone"],
             color=['#F4A7A7', 'red', 'yellow'])

ax.set_ylabel(r'RMSD ($\AA$)')
plt.savefig('RMSD_Afocus.png')

In [ ]:
# rmsd of cys and c loops of proe subunit
Cys_loop_E = "backbone and segid PROE and resid 133-147"
C_loop_E = "backbone and segid PROE and resid 192-200"

R = rms.RMSD(mobile, ref, select='backbone', groupselections=[Cys_loop_E, C_loop_E],ref_frame = 0)
R.run()

In [ ]:
df = pd.DataFrame(R.results.rmsd, columns=['Frame', 'Time (ps)', 'Backbone', 'Cys_loop_E', 'C_loop_E'])
df['Time (ns)']=(df['Time (ps)']/1000)
df

In [ ]:
ax = df.plot(x='Time (ns)',
             y=['Cys_loop_E', 'C_loop_E', "Backbone"],
             label =['Cys loop ' r'$\beta$/$\alpha$ E',  'C loop ' r'$\beta$/$\alpha$ E', "Backbone"],
             color=['#B0F4A7', 'green', 'yellow'])
ax.set_ylabel(r'RMSD ($\AA$)')
plt.savefig('RMSD_Efocus.png')

In [ ]:
# rmsd of cys and c loops of prod subunit
Cys_loop_D = "backbone and segid PROD and resid 131-145"
C_loop_D = "backbone and segid PROD and resid 192-199"

R = rms.RMSD(mobile, ref, select='backbone', groupselections=[Cys_loop_D, C_loop_D],ref_frame = 0)
R.run()

In [ ]:
df = pd.DataFrame(R.results.rmsd, columns=['Frame', 'Time (ps)', 'Backbone', 'Cys_loop_D', 'C_loop_D'])
df['Time (ns)']=(df['Time (ps)']/1000)
df

In [ ]:
ax = df.plot(x='Time (ns)',
             y=['Cys_loop_D', 'C_loop_D', "Backbone"],
             label =['Cys loop ' r'$\alpha$/$\beta$ D',  'C loop ' r'$\alpha$/$\beta$ D', "Backbone"],
             color=['#62DDE9', 'blue', 'yellow'])
ax.set_ylabel(r'RMSD ($\AA$)')
plt.savefig('RMSD_Dfocus.png')

In [ ]:
# rmsd of cys and c loops of prob subunit
Cys_loop_B = "backbone and segid PROB and resid 133-147"
C_loop_B = "backbone and segid PROB and resid 192-200"

R = rms.RMSD(mobile, ref, select='backbone', groupselections=[Cys_loop_B, C_loop_B],ref_frame = 0)
R.run()

In [ ]:
df = pd.DataFrame(R.results.rmsd, columns=['Frame', 'Time (ps)', 'Backbone', 'Cys_loop_B', 'C_loop_B'])
df['Time (ns)']=(df['Time (ps)']/1000)
df

In [ ]:
ax = df.plot(x='Time (ns)',
             y=['Cys_loop_B', 'C_loop_B', "Backbone"],
             label =['Cys loop ' r'$\beta$/$\beta$ B',  'C loop ' r'$\beta$/$\beta$ B', "Backbone"],
             color=['#F2C47B', 'orange', 'yellow'])

ax.set_ylabel(r'RMSD ($\AA$)')
plt.savefig('RMSD_Bfocus.png')

In [ ]:
# rmsd of c and cys loops of proc subunit
Cys_loop_C = "backbone and segid PROC and resid 133-147"
C_loop_C = "backbone and segid PROC and resid 192-200"

R = rms.RMSD(mobile, ref, select='backbone', groupselections=[Cys_loop_C, C_loop_C],ref_frame = 0)
R.run()

In [ ]:
df = pd.DataFrame(R.results.rmsd, columns=['Frame', 'Time (ps)', 'Backbone', 'Cys_loop_C', 'C_loop_C'])
df['Time (ns)']=(df['Time (ps)']/1000)
df

In [ ]:
ax = df.plot(x='Time (ns)',
             y=['Cys_loop_C', 'C_loop_C', "Backbone"],
             label =['Cys loop ' r'$\beta$/$\alpha$ C',  'C loop ' r'$\beta$/$\alpha$ C', "Backbone"],
             color=['#D87BF2', 'purple', 'yellow'])
ax.set_ylabel(r'RMSD ($\AA$')
plt.savefig('RMSD_Cfocus.png')

RMSD Done

RMSF ----------------------------------------------------------

In [ ]:
# select atoms on proa subunit to do rmsf
c_alphas_A = mobile.select_atoms('protein and name CA and segid PROA')
RfA = rms.RMSF(c_alphas_A).run()

In [ ]:
# make plot
plt.plot(c_alphas_A.resids, RfA.results.rmsf, color='red')
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')

# indicate where the c loop is
plt.axvspan(192, 199, zorder=0, alpha=0.2, color='orange', label='C loop')

plt.legend()
plt.savefig('RMSF_Afocus.png')

In [ ]:
# prod
c_alphas_D = mobile.select_atoms('protein and name CA and segid PROD')
RfD = rms.RMSF(c_alphas_D).run()

In [ ]:
plt.plot(c_alphas_D.resids, RfD.results.rmsf,color='green')
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')

# indicate where the c loop is
plt.axvspan(192, 199, zorder=0, alpha=0.2, color='orange', label='C loop')

plt.legend()
plt.savefig('RMSF_Dfocus.png')

In [ ]:
# proe
c_alphas_E = mobile.select_atoms('protein and name CA and segid PROE')
RfE = rms.RMSF(c_alphas_E).run()

In [ ]:
plt.plot(c_alphas_E.resids, RfE.results.rmsf,color='blue')
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')

# indicate where the c loop is
plt.axvspan(192, 200, zorder=0, alpha=0.2, color='orange', label='C loop')

plt.legend()
plt.savefig('RMSF_Efocus.png')

In [ ]:
# prob
c_alphas_B = mobile.select_atoms('protein and name CA and segid PROB')
RfB = rms.RMSF(c_alphas_B).run()

In [ ]:
plt.plot(c_alphas_B.resids, RfB.results.rmsf,color='orange')
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')

# indicate where the c loop is
plt.axvspan(192, 200, zorder=0, alpha=0.2, color='orange', label='C loop')

plt.legend()
plt.savefig('RMSF_Bfocus.png')

In [ ]:
# proc
c_alphas_C = mobile.select_atoms('protein and name CA and segid PROC')
RfC = rms.RMSF(c_alphas_C).run()

In [ ]:
plt.plot(c_alphas_C.resids, RfC.results.rmsf,color='purple')
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')

# indicate where the c loop is
plt.axvspan(192, 200, zorder=0, alpha=0.2, color='orange', label='C loop')

plt.legend()
plt.savefig('RMSF_Cfocus.png')

In [ ]:
# overlap binding site plots
plt.plot(c_alphas_A.resids, RfA.results.rmsf,color='red', label='C loop ' r'$\alpha$/$\beta$ A')
plt.plot(c_alphas_D.resids, RfD.results.rmsf,color='green', label='C loop ' r'$\alpha$/$\beta$ D')
plt.plot(c_alphas_E.resids, RfE.results.rmsf,color='blue', label='C loop ' r'$\beta$/$\alpha$ E')
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')
plt.axvspan(192, 200, zorder=0, alpha=0.2, color='green', label='C loop area')
plt.axvspan(131, 147, zorder=0, alpha=0.2, color='orange', label='Cys loop area')
plt.legend(loc='center right', bbox_to_anchor=(0.59, 0.8))
plt.savefig('RMSF_binding_sites.png')

In [ ]:
# overlap all site plots
plt.plot(c_alphas_A.resids, RfA.results.rmsf,color='red', label='C loop ' r'$\alpha$/$\beta$ A')
plt.plot(c_alphas_D.resids, RfD.results.rmsf,color='green', label='C loop ' r'$\alpha$/$\beta$ D')
plt.plot(c_alphas_E.resids, RfE.results.rmsf,color='blue', label='C loop ' r'$\beta$/$\alpha$ E')
plt.plot(c_alphas_B.resids, RfB.results.rmsf,color='orange', label='C loop ' r'$\beta$/$\beta$ B')
plt.plot(c_alphas_C.resids, RfC.results.rmsf,color='purple', label='C loop ' r'$\beta$/$\alpha$ C')
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')
plt.axvspan(192, 200, zorder=0, alpha=0.2, color='green', label='C loop area')
plt.axvspan(131, 147, zorder=0, alpha=0.2, color='orange', label='Cys loop area')
plt.legend(loc='upper center', bbox_to_anchor=(0.4,1))
plt.savefig('RMSF_all_sites.png')

In [ ]:
# focus on cys loop area of all subunits
plt.plot(c_alphas_A.resids, RfA.results.rmsf,color='red', label='C loop ' r'$\alpha$/$\beta$ A')
plt.plot(c_alphas_D.resids, RfD.results.rmsf,color='green', label='C loop ' r'$\alpha$/$\beta$ D')
plt.plot(c_alphas_E.resids, RfE.results.rmsf,color='blue', label='C loop ' r'$\beta$/$\alpha$ E')
plt.plot(c_alphas_B.resids, RfB.results.rmsf,color='orange', label='C loop ' r'$\beta$/$\beta$ B')
plt.plot(c_alphas_C.resids, RfC.results.rmsf,color='purple', label='C loop ' r'$\beta$/$\alpha$ C')
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')
plt.legend(loc='upper center', bbox_to_anchor=(0.2,1))
plt.axvspan(133, 147, zorder=0, alpha=0.2, color='green', label='' r'$\beta$ Cys loop')
plt.axvspan(131, 145, zorder=0, alpha=0.2, color='orange', label='' r'$\alpha$ Cys loop')

plt.xlim(right=147.5)
plt.xlim(left=130.5)

plt.savefig('RMSF_all_Cysloopfocus.png')

In [ ]:
# focus on c loop area of all subunits
plt.plot(c_alphas_A.resids, RfA.results.rmsf,color='red', label='C loop ' r'$\alpha$/$\beta$ A')
plt.plot(c_alphas_D.resids, RfD.results.rmsf,color='green', label='C loop ' r'$\alpha$/$\beta$ D')
plt.plot(c_alphas_E.resids, RfE.results.rmsf,color='blue', label='C loop ' r'$\beta$/$\alpha$ E')
plt.plot(c_alphas_B.resids, RfB.results.rmsf,color='orange', label='C loop ' r'$\beta$/$\beta$ B')
plt.plot(c_alphas_C.resids, RfC.results.rmsf,color='purple', label='C loop ' r'$\beta$/$\alpha$ C')
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')
plt.legend(loc='upper center', bbox_to_anchor=(0.2,1))
plt.axvspan(192, 200, zorder=0, alpha=0.2, color='green', label='' r'$\beta$ C loop')
plt.axvspan(192, 199, zorder=0, alpha=0.2, color='orange', label='' r'$\alpha$ C loop')

plt.xlim(right=200.5)
plt.xlim(left=190.5)

plt.savefig('RMSF_all_Cloopfocus.png')

RMSF done

DISTANCE -----------------------------------------------

In [ ]:
mobile.trajectory[-1] # set mobile trajectory to last frame
ref.trajectory[0] # set reference trajectory to first frame

# we want to look at the distance between the tip of the beta c loop and the
# backside tryptophan
principal_residue = u.select_atoms('name CA and segid PROE and resid 195 to 197')
complementary_residue = u.select_atoms('name CA and segid PROE and resid 154')

entry=[] # temporary variable to add an entry to our list of distances
i = 0 # counter variable (keeps track of the timestep)
dists=[] # our final list of distances (we will add to this)

# for every time step, calculate the distance
for ts in u.trajectory:
    # get the center of mass for each residue (group)
    cm_principal = principal_residue.center_of_mass()
    cm_complementary = complementary_residue.center_of_mass()
    
    # calculate the difference between the positions to get the distance
    difference = np.linalg.norm(cm_principal - cm_complementary)
    
    # create an entry with the timestep and distance
    entry = [i, difference]
    
    # add entry to our list
    dists.append(entry)
    # increment our counter variable for the next time step
    i += 1

In [ ]:
# create a data table
df = pd.DataFrame(dists, columns=['Frame','Distance'])
df['Time (ns)']=(df['Frame']/10)
df

In [ ]:
# create a plot
ax = df.plot(x='Time (ns)',
             y=['Distance'],
             color=['#6AD18A'])
ax.set_ylabel('Distance')
plt.savefig('Distance_allostericCloopatE.png')

In [ ]:
mobile.trajectory[-1] # set mobile trajectory to last frame
ref.trajectory[0] # set reference trajectory to first frame

# we want to look at the distance between the vicinal cysteines and the
# backside tryptophan
principal_residue = u.select_atoms('name CA and segid PROD and resid 195 to 196')
complementary_residue = u.select_atoms('name CA and segid PROD and resid 152')

entry=[] # temporary variable to add an entry to our list of distances
i = 0 # counter variable (keeps track of the timestep)
dists=[] # our final list of distances (we will add to this)

# for every time step, calculate the distance
for ts in u.trajectory:
    # get the center of mass for each residue (group)
    cm_principal = principal_residue.center_of_mass()
    cm_complementary = complementary_residue.center_of_mass()
    
    # calculate the difference between the positions to get the distance
    difference = np.linalg.norm(cm_principal - cm_complementary)
    
    # create an entry with the timestep and distance
    entry = [i, difference]
    
    # add entry to our list
    dists.append(entry)
    # increment our counter variable for the next time step
    i += 1

In [ ]:
# create a data table
df = pd.DataFrame(dists, columns=['Frame','Distance'])
df['Time (ns)']=(df['Frame']/10)
df

In [ ]:
# create a plot
ax = df.plot(x='Time (ns)',
             y=['Distance'],
             color=['#DBADCC'])
ax.set_ylabel('Distance')
plt.savefig('Distance_orthostericCloopatD.png')

In [ ]:
mobile.trajectory[-1] # set mobile trajectory to last frame
ref.trajectory[0] # set reference trajectory to first frame

# we want to look at the distance between the vicinal cysteines and the
# backside tryptophan
principal_residue = u.select_atoms('name CA and segid PROA and resid 195 to 196')
complementary_residue = u.select_atoms('name CA and segid PROA and resid 152')

entry=[] # temporary variable to add an entry to our list of distances
i = 0 # counter variable (keeps track of the timestep)
dists=[] # our final list of distances (we will add to this)

# for every time step, calculate the distance
for ts in u.trajectory:
    # get the center of mass for each residue (group)
    cm_principal = principal_residue.center_of_mass()
    cm_complementary = complementary_residue.center_of_mass()
    
    # calculate the difference between the positions to get the distance
    difference = np.linalg.norm(cm_principal - cm_complementary)
    
    # create an entry with the timestep and distance
    entry = [i, difference]
    
    # add entry to our list
    dists.append(entry)
    # increment our counter variable for the next time step
    i += 1

In [ ]:
# create a data table
df = pd.DataFrame(dists, columns=['Frame','Distance'])
df['Time (ns)']=(df['Frame']/10)
df

In [ ]:
# create a plot
ax = df.plot(x='Time (ns)',
             y=['Distance'],
             color=['#ADDBD4'])
ax.set_ylabel('Distance')
plt.savefig('Distance_orthostericCloopatA.png')